Instagram almacena en un RDD (id_user, nickname,pais,fecha_alta) y en otro RDD (id_publicacion, id_user, comentario, id_foto, fecha)

### Encontrar el top10 de hashtags que más aparecen y que fueron publicados por usuarios de Argentina

In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
usuarios_data = [
    (1, "Leo", "Argentina", 2015),
    (2, "Leo", "Argentina", 2014),
    (3, "Leo", "Brasil", 2013),
    (4, "Leo", "Argentina", 2012),
    (5, "Leo", "Brasil", 2014),
    (6, "Leo", "Argentina", 2016),
    (7, "Leo", "Bolivia", 2017),
    (8, "Leo", "Brasil", 2016)
]

usuarios_rdd = sc.parallelize(usuarios_data)

In [3]:
publicaciones_data = [
    (0, 1, "#please #hola #boosting publicacion", 1, 2019),
    (1, 2, "#leo #hola #hola #hola #freeky #freeky ,hola public, holahola", 2, 2019),
    (2, 3, "#chau #chau #chau #chau #chau #chau testinggg", 3, 2019),
    (3, 2, "no se que poner ni idea #drugs #", 2, 2019),
    (4, 5, "#Drugs #Vans #Nike", 3, 2018),
    (5, 6, "#Drugs #vans #professional", 10, 2018),
    (6, 6, "#messi #ronaldo #tesla", 20, 2019),
    (7, 8, "#messi messi messi #ronaldo push", 21, 2018),
    (8, 8, "#Nada #hola nada Hola hola", 18, 2019)
]

publicaciones_rdd = sc.parallelize(publicaciones_data)

In [4]:
usuarios_argentina = usuarios_rdd.filter(lambda x: x[2] == "Argentina").map(lambda x: (x[0], None))

In [5]:
publicaciones = publicaciones_rdd.map(lambda x: (x[1], x[2])) #los registros seran de la forma (id_user, comment)

In [6]:
#luego del join que se realiza abajo tenemos registros de la forma (user_id, (None, comentario_foto))

In [7]:
registros_palabras = usuarios_argentina.join(publicaciones)\
.flatMap(lambda x: x[1][1].split()) 

#ahora tenemos todas las palabras spliteadas, solo resta ver cuales empiezan con '#'

In [8]:
registros_palabras.filter(lambda x: x[0] == '#')\
.map(lambda x: (x,1))\
.reduceByKey(lambda x,y: x + y)\
.takeOrdered(10, key = lambda x: -x[1])

[('#hola', 4),
 ('#freeky', 2),
 ('#please', 1),
 ('#messi', 1),
 ('#tesla', 1),
 ('#leo', 1),
 ('#boosting', 1),
 ('#Drugs', 1),
 ('#', 1),
 ('#drugs', 1)]

### Encontrar el top10 de hashtags que más aparecen y que fueron publicados unicamente(!!) por usuarios de Argentina

De https://github.com/vaquarkhan/Apache-Kafka-poc-and-notes/wiki/Difference-between-flatMap()-and-map()-on-an-RDD podemos concluir que lo que la idea de flatmap es para cada registro del RDD aplicar una funcion de la forma: lambda x: function(x) donde function devuelve una lista de N elementos. El resultado del flatMap es un nuevo registro por cada elemento de la lista que surge de aplicar la funcion a cada elemento del RDD.

In [9]:
publicaciones_rdd.flatMap(lambda x: [(x[1],x[2].split())]).collect()
#notar que hacer esto es lo mismo q hacer publicaciones_rdd.map(lambda x: (x[1],x[2].split())).collect()

[(1, ['#please', '#hola', '#boosting', 'publicacion']),
 (2,
  ['#leo',
   '#hola',
   '#hola',
   '#hola',
   '#freeky',
   '#freeky',
   ',hola',
   'public,',
   'holahola']),
 (3, ['#chau', '#chau', '#chau', '#chau', '#chau', '#chau', 'testinggg']),
 (2, ['no', 'se', 'que', 'poner', 'ni', 'idea', '#drugs', '#']),
 (5, ['#Drugs', '#Vans', '#Nike']),
 (6, ['#Drugs', '#vans', '#professional']),
 (6, ['#messi', '#ronaldo', '#tesla']),
 (8, ['#messi', 'messi', 'messi', '#ronaldo', 'push']),
 (8, ['#Nada', '#hola', 'nada', 'Hola', 'hola'])]

In [10]:
def find_hashtags_users(register):
    #we want to return something like return [(id_user_i, "#hola"),(id_user_i, "#mood"),(id_user_i, "#goodbye"),..]
    comments = register[2].split()
    hashtags = []
    for elem in comments:
        if elem[0] == '#':
            hashtags.append((register[1], elem))
    return hashtags

In [11]:
user_id_hashtags = publicaciones_rdd.flatMap(lambda x: find_hashtags_users(x))

In [12]:
user_id_hashtags.collect()

[(1, '#please'),
 (1, '#hola'),
 (1, '#boosting'),
 (2, '#leo'),
 (2, '#hola'),
 (2, '#hola'),
 (2, '#hola'),
 (2, '#freeky'),
 (2, '#freeky'),
 (3, '#chau'),
 (3, '#chau'),
 (3, '#chau'),
 (3, '#chau'),
 (3, '#chau'),
 (3, '#chau'),
 (2, '#drugs'),
 (2, '#'),
 (5, '#Drugs'),
 (5, '#Vans'),
 (5, '#Nike'),
 (6, '#Drugs'),
 (6, '#vans'),
 (6, '#professional'),
 (6, '#messi'),
 (6, '#ronaldo'),
 (6, '#tesla'),
 (8, '#messi'),
 (8, '#ronaldo'),
 (8, '#Nada'),
 (8, '#hola')]

In [13]:
usuarios_no_argentinos = usuarios_rdd.filter(lambda x: x[2] != "Argentina").map(lambda x: (x[0], None))

In [14]:
hashtags_no_queridos = usuarios_no_argentinos.\
join(user_id_hashtags).map(lambda x: x[1][1]) #ya tenemos los hashtags que no queremos contabilizar

In [15]:
hashtags_no_queridos.collect()

['#chau',
 '#chau',
 '#chau',
 '#chau',
 '#chau',
 '#chau',
 '#Drugs',
 '#Vans',
 '#Nike',
 '#messi',
 '#ronaldo',
 '#Nada',
 '#hola']